In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns

In [2]:
#so all DataFrame data is displayed
pd.set_option('display.max_columns',None)
pd.set_option('display.min_rows',0)
pd.set_option('display.max_rows',100)

In [3]:
housing = pd.read_csv("./data/Ames_Housing_Price_Data.csv", index_col=0)
realestate = pd.read_csv('./data/Ames_Real_Estate_Data.csv', index_col=0)

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Getting the Latitude And Longitude

In [4]:
#making index a column 
realestate['MapRefNo'] = realestate.index

# creating a new dataframe with only necessary columns 
realestate1 = realestate[['MapRefNo', 'GeoRefNo', 'Prop_Addr', 'Neighborhood', 'ClassPr_S', 'SchD_S', 'TxD_S']]

#renamed realestate MapRefNo to PID to Merge on PID 
realestate1.rename(columns = {'MapRefNo':'PID'}, inplace = True) 

#Merging Dataframes on PID
clean_df = pd.merge(housing, realestate1, on = 'PID', how='left')


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
#dropping duplicates 
clean_df = clean_df.drop_duplicates(subset = ['PID'])
clean_df.shape #checking shape

(2579, 87)

In [6]:
# Function to get Latitude and Longitude

from geopy.geocoders import Nominatim
# location = geolocator.geocode("2400 SE 16TH ST AMES, IA")

def get_lat_log(address):
    geolocator = Nominatim(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")
    geocode = lambda query: geolocator.geocode("%s, AMES IA" % query)
    location = geocode(address)
    
    if location == None:
        return None 
    
    return (location.latitude, location.longitude)

In [7]:
# Function to get distance from schools
from geopy.distance import geodesic

def get_school_dist(lat_log, school_lat_log):
    """ Get School Distance
    :lat_log - tuple, address lat and log
    :school_lst_log -tuple, school lat and log
    Return: distance between both addresses
    """
    return(geodesic(lat_log, school_lat_log).miles)

In [8]:
from time import sleep


addresses = clean_df["Prop_Addr"]

total_addresses = len(addresses)
count = 0
none_count_hs = 0
none_count_c = 0

high_school = "Ames High School 1921 Ames High Drive Ames, IA 50010"
hs_lat_log = get_lat_log(high_school)

college = "100 Enrollment Services Ctr"
c_lat_log = get_lat_log(college)

address_lat_log = []
highschool_dist = []
college_dist = []

for address in addresses:
    sleep(.25)
    lat_log = get_lat_log(address)
    address_lat_log.append(lat_log)
    
    dist_hs = get_school_dist(lat_log, hs_lat_log)
    if dist_hs is None:
        dist_hs = 0
        non_count_hs += 1
    highschool_dist.append(dist_hs) 
    
    sleep(.25)
    dist_c = get_school_dist(lat_log, c_lat_log)
    if dist_c is None:
        dist_c = 0
        non_count_c += 1
    college_dist.append(dist_c)
    
    
    count += 1
    print(f"On address {count} out of {total_addresses}")
    print(f"Got None: High School: {none_count_hs} college: {none_count_c}")

print("Done!")


On address 1 out of 2579
Got None: High School: 0 college: 0
On address 2 out of 2579
Got None: High School: 0 college: 0
On address 3 out of 2579
Got None: High School: 0 college: 0
On address 4 out of 2579
Got None: High School: 0 college: 0
On address 5 out of 2579
Got None: High School: 0 college: 0
On address 6 out of 2579
Got None: High School: 0 college: 0
On address 7 out of 2579
Got None: High School: 0 college: 0
On address 8 out of 2579
Got None: High School: 0 college: 0
On address 9 out of 2579
Got None: High School: 0 college: 0
On address 10 out of 2579
Got None: High School: 0 college: 0
On address 11 out of 2579
Got None: High School: 0 college: 0
On address 12 out of 2579
Got None: High School: 0 college: 0
On address 13 out of 2579
Got None: High School: 0 college: 0
On address 14 out of 2579
Got None: High School: 0 college: 0
On address 15 out of 2579
Got None: High School: 0 college: 0
On address 16 out of 2579
Got None: High School: 0 college: 0
On address 17 out

KeyboardInterrupt: 

In [ ]:
#make list from functions into columns in df
clean_df["address_lat_log"] = address_lat_log
clean_df['highschool_dist'] = highschool_dist
clean_df['college_dist'] = college_dist

# separate tuple to two columns
clean_df[['latitude', 'longitude']] = pd.DataFrame(clean_df["address_lat_log"].tolist(), index = clean_df.index)


clean_df.shape

In [ ]:
#looking at zoning - Will only keep residential
clean_df.MSZoning.value_counts()

In [ ]:
#drop non residential properties 
clean_df = clean_df[clean_df.MSZoning != 'C (all)']
clean_df = clean_df[clean_df.MSZoning != 'I (all)']
clean_df = clean_df[clean_df.MSZoning != 'A (agr)']

In [ ]:
#save dataframe with lat-log and distances to not need to run above 
clean_df.to_pickle('clean_Res_latlog.pkl')

In [ ]:
#check 
clean_df.MSZoning.value_counts()

In [ ]:
# check to see what it looks like after dropping CIA 
clean_df['Functional'].value_counts()